In [0]:
import json
import pandas as pd
import re
import numpy as np
import string
pd.options.mode.chained_assignment = None

import spacy
from spacy import displacy
from collections import Counter
import en_core_web_sm
nlp = en_core_web_sm.load()

token = {"username":"your_username","key":"your_key"}   # Download kaggle.json file from your kaggle account and fill in your username and key
with open('/root/.kaggle/kaggle.json', 'w') as file:
    json.dump(token, file)

!kaggle datasets download -d wcukierski/enron-email-dataset -p /content
!unzip \*.zip

chunk = pd.read_csv('/content/emails.csv', chunksize=516000)
df = next(chunk)

def get_text(Series, row_num_slicer):
    """returns a Series with text sliced from a list split from each message. Row_num_slicer
    tells function where to slice split text to find only the body of the message."""
    result = pd.Series(index=Series.index)
    for row, message in enumerate(Series):
        message_words = message.split('\n')
        del message_words[:row_num_slicer]
        result.iloc[row] = message_words
    return result

def extract(st):
  match = re.search('Subject:(.+?)Thanks',st)
  if match:
    st = st[match.start():match.end()]
    return st
  else:
    return None

remove = string.punctuation
remove = remove.replace(":", "") # don't remove colons
pattern = r"[{}]".format(remove) # create the pattern

def rm_symbols(s):
  s = ''.join(re.sub(pattern, "", s))       
  return s

def rm_slash_t(s):
  s = ''.join(re.sub(r'\t','',s))       
  return s

def rm_colon_word(s):
  s = ''.join(re.sub(r'[A-Za-z0-9][A-Za-z0-9. ]*:','',s))       
  return s

def rm_colon(s):
  s = ''.join(re.sub(r':','',s))       
  return s

def rm_emailid(s):
  s = ''.join(re.sub(r'\S*@\S*\s?','',s))       
  return s

def rm_date(s):
  s = ''.join(re.sub(r'[0-9]{2}[\/,:][0-9]{2}[\/,:][0-9]{2,4}','',s))        
  return s

def rm_time(s):
  s = ''.join(re.sub(r'(1[0-2]|0?[1-9]):([0-5][0-9]) ([AaPp][Mm])','',s))       
  return s

def rm_file_name(s):
  s = ''.join(re.sub(r'/[^\\]*\.(\w+)$/','',s))       
  return s

def rm_digit(s):
  s = ''.join([i for i in s if not i.isdigit()])
  return s

def rm_url(s):
  s = ''.join(re.sub(r'http\S+','',s))
  return s

def rm_spaces(s):
  s = ' '.join(s.split())
  return s

def rm_slash(s):
  s = ''.join(re.sub(r'\S*/\S*\s?','',s))   
  return s

def rm_tab(s):
  s = ''.join(re.sub(r'\S*\\S*\s?','',s))   
  return s

def remove_ent(s):
  doc = nlp(s)
  print(type(doc))
  print([(X.text, X.label_) for X in doc.ents])
  text_no_namedentities = ""
  for token in doc:
    if not token.ent_type:
      text_no_namedentities += token.text
      if token.whitespace_:
        text_no_namedentities += " "
  return text_no_namedentities

df['text'] = get_text(df.message, 15)

df = df[['text']]

df['text'] = df['text'].apply(str)
df['text'] = df['text'].apply(extract)

df = df.replace(to_replace='None', value=np.nan).dropna()

df['text'] = df['text'].apply(rm_slash_t)
df['text'] = df['text'].apply(rm_emailid)
df['text'] = df['text'].apply(rm_date)
df['text'] = df['text'].apply(rm_time)
df['text'] = df['text'].apply(rm_digit)
df['text'] = df['text'].apply(rm_url)
df['text'] = df['text'].apply(rm_symbols)
df['text'] = df['text'].apply(rm_slash)
df['text'] = df['text'].apply(rm_file_name)
df['text'] = df['text'].apply(rm_tab)
df['text'] = df['text'].apply(rm_colon_word)
df['text'] = df['text'].apply(rm_colon)
df['text'] = df['text'].apply(rm_spaces)

df['text'] = df['text'].apply(remove_ent)
df['text'] = df['text'].apply(remove_ent)
df['text'] = df['text'].apply(remove_ent)


df = df[df.text != 'Thanks']  
df.dropna
df.to_csv('entities_removed2.csv', index= False)